### Taxonomic Profiling on MAGS
- Can't figure out on anvio so for now trying other options
- option 1: sylph 
- option 2: kraken

#### Sylph
https://github.com/bluenote-1577/sylph
- average nucleic identity (ANI) querying or taxonomic profiling

In [ ]:
#activate unity interactive job..need high memory for installation
salloc -p cpu --mem 150G

In [ ]:
%conda init --all

In [ ]:
#module load miniconda/22.11.1-1
#conda create -n taxa
%conda activate taxa

In [ ]:
#install sylph package
conda install -c bioconda sylph
#if above doesnt work do: 
conda install -c bioconda -c conda-forge sylph

In [ ]:
#set parameters
SAMPLENAME=mcav
CONTIGPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/results/mcav_assembly3
CONTIGFILE=mcav.contigs-fixed.fa

In [ ]:
#sketch paired-end reads
sylph sketch -1 ./trimmed/*_R1_001_val_1.fq -2 ./trimmed/*_R2_001_val_2.fq -d "$SAMPLENAME"_sketches -t 50 
#decrease -c (default =200)...

In [ ]:
#create database if contigs are all in one fasta file
sylph sketch $CONTIGPATH/$CONTIGFILE -i -o ./working/"$SAMPLENAME"_contig_database

In [ ]:
# query for ANI 
sylph query "$SAMPLENAME"_sketches/*.sylsp ./working/"$SAMPLENAME"_contig_database.syldb
#output is statistics for each genome against our reads
#redirect output??
#Thus sylph query is not a profiler. It does not tell you the abundance of the genomes in your sample, just how similar your query genome is to the genomes within the sample.

In [ ]:
#Import taxa database: https://github.com/bluenote-1577/sylph/wiki/Pre%E2%80%90built-databases
wget https://storage.googleapis.com/sylph-stuff/v0.3-c200-gtdb-r214.syldb -O gtdb_database.syldb

In [ ]:
#move database into folder called gtdb_genomes_reps_r214
mkdir working/gtdb_genomes_reps_r214
mv gtdb_database.syldb working/gtdb_genomes_reps_r214/gtdb_database.syldb

#set path to GTDB
DB=working/gtdb_genomes_reps_r214/gtdb_database.syldb
#set input & output path
INDIR=mcav_sketches
OUTDIR=results/taxonomy

In [ ]:
#maps samples against contig database???
#read more: https://github.com/bluenote-1577/sylph/wiki/sylph-cookbook#creating-a-database-of-fasta-files
sylph profile "$SAMPLENAME"_sketches/*.sylsp ./working/"$SAMPLENAME"_contig_database.syldb -t 50 -o ./results/"$SAMPLENAME"_sylph_profile.tsv

In [ ]:
#map reads against GTDB 
sylph profile $INDIR/*_MCAV_R1_001_val_1.fq.paired.sylsp "$DB" -t 10 -o $OUTDIR/sylph_GTDB.tsv

In [ ]:
#get taxonomic profile 
#clone repo (in "working" dir)
git clone https://github.com/bluenote-1577/sylph
python working/sylph/scripts/sylph_to_taxprof.py -s $OUTDIR/sylph_GTDB.tsv -m  working/sylph/scripts/gtdb_r214_metadata.tsv.gz -o $OUTDIR/prefix_.tsv

#results: only 6 samples have hits on the reference genomes database

In [ ]:
#convert to tsv

# Loop over all files 
for input_file in prefix_*_R1_001_val_1.fq.sylphmpa; do
    # Extract the unique identifier from the file name
    unique_identifier=$(echo "$input_file" | sed 's/^prefix_\(.*\)_R1_001_val_1\.fq\.sylphmpa$/\1/')

    # Generate the output TSV file name
    output_file="${unique_identifier}.tsv"

    # Process the input file and save the result to the output file
    awk -F'\t' '{for (i=1; i<=NF; i++) printf "%s%s", $i, (i<NF ? "\t" : "\n")}' "$input_file" > "$output_file"

    echo "Processed $input_file and saved as $output_file"
done


In [ ]:
#repeating with OceanDNA database
wget https://storage.googleapis.com/sylph-stuff/OceanDNA-c200-v0.3.syldb
mv OceanDNA-c200-v0.3.syldb working/gtdb_genomes_reps_r214/OceanDNA-c200-v0.3.syldb
oceanDB=working/gtdb_genomes_reps_r214/OceanDNA-c200-v0.3.syldb

sylph profile $INDIR/*_MCAV_R1_001_val_1.fq.paired.sylsp "$oceanDB" -t 10 -o $OUTDIR/sylph_ocean.tsv

#### Kraken
- https://ccb.jhu.edu/software/kraken/MANUAL.html

having trouble building "standard" databases...using silva anyway
- ask unity?
https://bioinformaticsworkbook.org/dataAnalysis/Metagenomics/Kraken.html#gsc.tab=0

https://bioinformatics.stackexchange.com/questions/20589/kraken2-standard-database-failing-to-build-unexpected-ftp-path?rq=1

https://github.com/DerrickWood/kraken2/issues/292

In [ ]:
module load miniconda/22.11.1-1
module load kraken2/2.1.2

In [ ]:
# download SILVA database
wget -c url https://genome-idx.s3.amazonaws.com/kraken/16S_Silva132_20200326.tgz
#more db options (thinking about 'standard': https://benlangmead.github.io/aws-indexes/k2
# unzip 
tar -xvzf //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_Silva132_20200326.tgz

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-kraken-%j.out  # %j = job ID

module load miniconda/22.11.1-1
module load kraken2/2.1.2

# i think this specifically downloads NCBI and RefSeq
#kraken2-build --build --standard --threads 24 --db $DBNAME
#error with std db: kraken2: database ("//project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard") does not contain necessary file taxo.k2d
#/var/spool/slurm/slurmd/job15982456/slurm_script: line 23: --report: command not found
# see error notes above

# set variables
DBNAME=//project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db
READSPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken

#mkdir /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy
#mkdir /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken
# classify each set of paired end reads against silva taxonomic database
for f in T1_12_2022 T1_13_2022 T1_16_2019 T1_20_2019 T1_24_2019 T1_40_2022 T1_57_2022 T1_70_2022 T2_10_2022 T2_16_2019 T3_13_2022 T3_14_2019 T3_15_2019 T3_19_2022 T3_1_2019 T3_40_2022 T3_48_2022 T3_49_2022 T3_51_2022 T3_60_2022 T3_8_2019 T3_9_2019
do
kraken2 --db $DBNAME --threads 4 --report $OUTDIR/${f}.kreport2 --paired $READSPATH/${f}_MCAV_R1_001_val_1.fq $READSPATH/${f}_MCAV_R2_001_val_2.fq > $OUTDIR/${f}.kraken2
done

#bash script: kraken
#JOB ID: 16083608
# SUCCESS!

##### BRACKEN
- est species abundances??
https://github.com/jenniferlu717/Bracken/blob/master/README.md

In [ ]:
# Installation

#create symlink
ln -s //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts  //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke
ln -s //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke
#execute bracken scripts from second dir path 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-bracken-%j.out  # %j = job ID

module load miniconda/22.11.1-1
module activate kraken
module load kraken2/2.1.2

DBNAME=//project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db
THREADS=3
KMER_LEN=35
READ_LEN=250
BRACKEN_BUILD_PATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts
# Generate bracken database - just do once
#$BRACKEN_BUILD_PATH/bracken-build.txt -d ${DBNAME} -t ${THREADS} -k ${KMER_LEN} -l ${READ_LEN} 
#bracken-build -d ${DBNAME} -k ${KMER_LEN} -l ${READ_LEN} 
#${KRAKEN_DB}`  = location of the built Kraken 1/Kraken 2/KrakenUniq database
#${THREADS}`    = number of threads to use with Kraken and the Bracken scripts
#${KMER_LEN}`   = length of kmer used to build the Kraken database 
    #Kraken 1/KrakenUniq default kmer length = 31
    #Kraken 2 default kmer length = 35
    #Default set in the script is 35. 
#${READ_LEN}`   = the read length of your data e.g., if you are using 100 bp reads, set it to `100`. 
#need taxo.k2d, hash.k2d, and opts.k2d. in kraken2 database

# Abundance Estimation
FILEPATH=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken
#ls $FILEPATH | find *.kreport2 > kraken_list.txt
#cat kraken_list.txt| sed 's/.kreport2/''/' > SAMPLEID
SCRIPTPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/KrakenTools-master/DiversityTools
#$BRACKEN_BUILD_PATH/bash install_bracken.sh

while IFS= read -r line; do
    # Execute bracken command for each file
    bracken -d "$DBNAME" -i $FILEPATH/"$line" -o "$FILEPATH/${line%.kreport2}.bracken" -r "$READ_LEN" -l 'G'
done < $FILEPATH/kraken_list.txt
#${SAMPLE}.kreport - the kraken report generated for a given dataset
#{BRACKEN_OUTPUT_FILE}.bracken - the desired name of the output file to be generated by the code
#The following optional parameters may be specified:

#${LEVEL} - Default = 'S'. This specifies that abundance estimation will calculate estimated reads for each species. Other possible options are K (kingdom level), P (phylum), C (class), O (order), F (family), and G (genus).
#${THRESHOLD} - Default = 10. For species classification, any species with <= 10 (or otherwise specified) reads will not receive any additional reads from higher taxonomy levels when distributing reads for abundance estimation. 
#If another classification level is specified, thresholding will occur at that level.

#repeating for speceis just for funsies
while IFS= read -r line; do
    # Execute bracken command for each file
    bracken -d "$DBNAME" -i $FILEPATH/"$line" -o "$FILEPATH/${line%.kreport2}_species.bracken" -r "$READ_LEN"
done < kraken_list.txt

#bash script: bracken_script 
#JOB ID: 16083669

###### Genus & Species output


ROGRAM START TIME: 12-14-2023 02:40:25
BRACKEN SUMMARY (Kraken report: T1_12_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 20742228
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 48519
          >> Unclassified reads: 20693709
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_12_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_13_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_13_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:26
BRACKEN SUMMARY (Kraken report: T1_13_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 20861960
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 50615
          >> Unclassified reads: 20811345
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_13_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_16_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_16_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:26
BRACKEN SUMMARY (Kraken report: T1_16_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 25807031
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 71664
          >> Unclassified reads: 25735367
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_16_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_20_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_20_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_20_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 8517445
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 44133
          >> Unclassified reads: 8473312
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_20_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:27
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_24_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_24_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_24_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 17135017
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 59096
          >> Unclassified reads: 17075921
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_24_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:27
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_40_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_40_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_40_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15419779
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 110723
          >> Unclassified reads: 15309056
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_40_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:28
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_57_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_57_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:28
BRACKEN SUMMARY (Kraken report: T1_57_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 31964707
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 157041
          >> Unclassified reads: 31807666
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_57_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:28
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_70_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_70_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:28
BRACKEN SUMMARY (Kraken report: T1_70_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15776674
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 121001
          >> Unclassified reads: 15655673
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_70_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:29
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T2_10_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_10_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:29
BRACKEN SUMMARY (Kraken report: T2_10_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 16876812
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 57221
          >> Unclassified reads: 16819591
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_10_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:29
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T2_16_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_16_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:29
BRACKEN SUMMARY (Kraken report: T2_16_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 34302369
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 116886
          >> Unclassified reads: 34185483
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_16_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:30
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_13_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_13_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:30
BRACKEN SUMMARY (Kraken report: T3_13_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13449647
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 41610
          >> Unclassified reads: 13408037
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_13_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:30
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_14_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_14_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:30
BRACKEN SUMMARY (Kraken report: T3_14_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 21400877
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 91615
          >> Unclassified reads: 21309262
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_14_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_15_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_15_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_15_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 19854618
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 51464
          >> Unclassified reads: 19803154
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_15_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_19_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_19_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_19_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 9044371
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 93872
          >> Unclassified reads: 8950499
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_19_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_1_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_1_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_1_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 29977945
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 99522
          >> Unclassified reads: 29878423
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_1_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:32
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_40_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_40_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:32
BRACKEN SUMMARY (Kraken report: T3_40_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 26755227
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 77871
          >> Unclassified reads: 26677356
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_40_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:32
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_48_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_48_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:32
BRACKEN SUMMARY (Kraken report: T3_48_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13255150
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 133945
          >> Unclassified reads: 13121205
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_48_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:33
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_49_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_49_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:33
BRACKEN SUMMARY (Kraken report: T3_49_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15749618
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 149084
          >> Unclassified reads: 15600534
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_49_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:33
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_51_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_51_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:33
BRACKEN SUMMARY (Kraken report: T3_51_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13877944
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 50639
          >> Unclassified reads: 13827305
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_51_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_8_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_8_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:34
BRACKEN SUMMARY (Kraken report: T3_8_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 16453406
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 94662
          >> Unclassified reads: 16358744
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_8_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_9_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_9_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:34
BRACKEN SUMMARY (Kraken report: T3_9_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 14032207
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 69979
          >> Unclassified reads: 13962228
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_9_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.

Species output 
ROGRAM START TIME: 12-14-2023 02:40:25
BRACKEN SUMMARY (Kraken report: T1_12_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 20742228
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 48519
          >> Unclassified reads: 20693709
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_12_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_13_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_13_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:26
BRACKEN SUMMARY (Kraken report: T1_13_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 20861960
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 50615
          >> Unclassified reads: 20811345
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_13_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_16_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_16_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:26
BRACKEN SUMMARY (Kraken report: T1_16_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 25807031
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 71664
          >> Unclassified reads: 25735367
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_16_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:26
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_20_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_20_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_20_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 8517445
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 44133
          >> Unclassified reads: 8473312
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_20_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:27
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_24_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_24_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_24_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 17135017
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 59096
          >> Unclassified reads: 17075921
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_24_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:27
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_40_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_40_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:27
BRACKEN SUMMARY (Kraken report: T1_40_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15419779
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 110723
          >> Unclassified reads: 15309056
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_40_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:28
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_57_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_57_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:28
BRACKEN SUMMARY (Kraken report: T1_57_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 31964707
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 157041
          >> Unclassified reads: 31807666
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_57_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:28
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T1_70_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_70_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:28
BRACKEN SUMMARY (Kraken report: T1_70_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15776674
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 121001
          >> Unclassified reads: 15655673
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T1_70_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:29
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T2_10_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_10_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:29
BRACKEN SUMMARY (Kraken report: T2_10_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 16876812
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 57221
          >> Unclassified reads: 16819591
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_10_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:29
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T2_16_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_16_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:29
BRACKEN SUMMARY (Kraken report: T2_16_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 34302369
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 116886
          >> Unclassified reads: 34185483
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T2_16_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:30
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_13_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_13_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:30
BRACKEN SUMMARY (Kraken report: T3_13_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13449647
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 41610
          >> Unclassified reads: 13408037
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_13_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:30
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_14_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_14_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:30
BRACKEN SUMMARY (Kraken report: T3_14_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 21400877
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 91615
          >> Unclassified reads: 21309262
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_14_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_15_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_15_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_15_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 19854618
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 51464
          >> Unclassified reads: 19803154
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_15_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_19_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_19_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_19_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 9044371
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 93872
          >> Unclassified reads: 8950499
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_19_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:31
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_1_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_1_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:31
BRACKEN SUMMARY (Kraken report: T3_1_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 29977945
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 99522
          >> Unclassified reads: 29878423
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_1_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:32
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_40_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_40_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:32
BRACKEN SUMMARY (Kraken report: T3_40_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 26755227
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 77871
          >> Unclassified reads: 26677356
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_40_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:32
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_48_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_48_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:32
BRACKEN SUMMARY (Kraken report: T3_48_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13255150
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 133945
          >> Unclassified reads: 13121205
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_48_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:33
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_49_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_49_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:33
BRACKEN SUMMARY (Kraken report: T3_49_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 15749618
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 149084
          >> Unclassified reads: 15600534
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_49_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:33
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_51_2022.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_51_2022_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:33
BRACKEN SUMMARY (Kraken report: T3_51_2022.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 13877944
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 50639
          >> Unclassified reads: 13827305
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_51_2022_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_8_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_8_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:34
BRACKEN SUMMARY (Kraken report: T3_8_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 16453406
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 94662
          >> Unclassified reads: 16358744
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_8_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.
 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i T3_9_2019.kreport2 -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_9_2019_species.bracken -k //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db/database250mers.kmer_distrib -l S -t 0
PROGRAM START TIME: 12-14-2023 02:40:34
BRACKEN SUMMARY (Kraken report: T3_9_2019.kreport2)
    >>> Threshold: 0 
    >>> Number of species in sample: 0 
          >> Number of species with reads > threshold: 0 
          >> Number of species with reads < threshold: 0 
    >>> Total reads in sample: 14032207
          >> Total reads kept at species level (reads > threshold): 0
          >> Total reads discarded (species reads < threshold): 0
          >> Reads distributed: 0
          >> Reads not distributed (eg. no species above threshold): 69979
          >> Unclassified reads: 13962228
BRACKEN OUTPUT PRODUCED: //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken/bracken/T3_9_2019_species.bracken
PROGRAM END TIME: 12-14-2023 02:40:34
  Bracken complete.

##### KrakenTools 
- downstream analysis using kraken results
- https://ccb.jhu.edu/software/krakentools/index.shtml
- download zip file 

In [ ]:
SCRIPTPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/KrakenTools-master/DiversityTools
cd $SCRIPTPATH
# make scripts executable 
chmod +x alpha\_diversity.py
./alpha\_diversity.py -h
# input file must be bracken

chmod +x beta\_diversity.py
./beta\_diversity.py -h
# Bray-Curtis dissimilarity

In [ ]:
# Alpha Diversity
# Shannon diversity - genus level 

SCRIPTPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/KrakenTools-master/DiversityTools
FILEPATH=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken
#ls $FILEPATH | find *.kreport2 > kraken_list.txt
cat kraken_list.txt| sed 's/.kreport2/''/' > SAMPLEID

while IFS= read -r line; do
    python $SCRIPTPATH/alpha_diversity.py -f ${line}.bracken 
done < SAMPLEID

# don't know if it does multiple at a time or just one at a time...and what is output?
#-f, --filename MYFILE.BRACKEN..........Bracken output file
#-a, --alpha TYPE.......................Single letter alpha diversity type (Sh, BP, Si, ISi, F)

Shannon's diversity: 2.8405589418696024
Shannon's diversity: 3.747956479976194
Shannon's diversity: 3.240773802595393
Shannon's diversity: 3.357943870893113
Shannon's diversity: 3.127505271223385
Shannon's diversity: 2.948836172095586
Shannon's diversity: 3.155355846512862
Shannon's diversity: 2.9534230075080843
Shannon's diversity: 3.837848266676804
Shannon's diversity: 3.017049434243576
Shannon's diversity: 3.3014806325360837
Shannon's diversity: 3.1828393709134533
Shannon's diversity: 2.673104478260622
Shannon's diversity: 2.8628687828400676
Shannon's diversity: 3.1197450036741374
Shannon's diversity: 3.4793331275395554
Shannon's diversity: 2.842877602269089
Shannon's diversity: 2.9019378226014787
Shannon's diversity: 3.1864428196047045
Shannon's diversity: 3.1428692882448113
Shannon's diversity: 3.0649759367640863

In [ ]:
cat SAMPLEID

T1_12_2022
T1_13_2022
T1_16_2019
T1_20_2019
T1_24_2019
T1_40_2022
T1_57_2022
T1_70_2022
T2_10_2022
T2_16_2019
T3_13_2022
T3_14_2019
T3_15_2019
T3_19_2022
T3_1_2019
T3_40_2022
T3_48_2022
T3_49_2022
T3_51_2022
T3_8_2019
T3_9_2019

In [ ]:
# Beta Diversity

# collect list of input files
FILEPATH=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken
mv *_species.bracken species
ls $FILEPATH | find *.bracken > bracken_list.txt
FILES=xargs -a bracken_list.txt -d '\n'

In [ ]:
python $SCRIPTPATH/beta_diversity.py --type bracken --input-files T1_12_2022.bracken T1_13_2022.bracken T1_16_2019.bracken T1_20_2019.bracken T1_24_2019.bracken T1_40_2022.bracken T1_57_2022.bracken T1_70_2022.bracken T2_10_2022.bracken T2_16_2019.bracken T3_13_2022.bracken T3_14_2019.bracken T3_15_2019.bracken T3_19_2022.bracken T3_1_2019.bracken T3_40_2022.bracken T3_48_2022.bracken T3_49_2022.bracken T3_51_2022.bracken T3_8_2019.bracken T3_9_2019.bracken > b_diversity.txt


##### Pavian

In [3]:
if (!require(remotes)) { install.packages("remotes") }
remotes::install_github("fbreitwieser/pavian")

Loading required package: remotes

Skipping install of 'pavian' from a github remote, the SHA1 (3cf9d6f8) has not changed since last install.
  Use `force = TRUE` to force installation



#### Attempting to use Skbio to create pcoa plot from b diversity matrix

- matrix is non symmetrical ...something is wrong with the output from krakentools...probably something wrong with the files is my guess

In [1]:
from skbio.diversity import beta_diversity
from skbio.stats.distance import mantel

In [3]:
#view b diversity file 
file_path = '//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav_taxonomy/kraken/b_diversity.txt'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)

#0	T1_12_2022.bracken (36778 reads)
#1	T1_13_2022.bracken (34252 reads)
#2	T1_16_2019.bracken (55664 reads)
#3	T1_20_2019.bracken (31290 reads)
#4	T1_24_2019.bracken (43148 reads)
#5	T1_40_2022.bracken (68425 reads)
#6	T1_57_2022.bracken (105963 reads)
#7	T1_70_2022.bracken (81828 reads)
#8	T2_10_2022.bracken (43785 reads)
#9	T2_16_2019.bracken (92140 reads)
#10	T3_13_2022.bracken (29579 reads)
#11	T3_14_2019.bracken (62084 reads)
#12	T3_15_2019.bracken (36798 reads)
#13	T3_19_2022.bracken (57643 reads)
#14	T3_1_2019.bracken (72214 reads)
#15	T3_40_2022.bracken (53906 reads)
#16	T3_48_2022.bracken (81024 reads)
#17	T3_49_2022.bracken (90950 reads)
#18	T3_51_2022.bracken (34308 reads)
#19	T3_8_2019.bracken (66802 reads)
#20	T3_9_2019.bracken (48412 reads)
x	0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20
0	0.000	0.453	0.225	0.426	0.180	0.414	0.500	0.466	0.330	0.471	0.262	0.310	0.163	0.413	0.345	0.390	0.501	0.500	0.243	0.459	0.251
1	x.xxx	0.000	0.437	0.628	0.472	0.669	0.625	0.692	0.

In [4]:
# Plan
# load in first part as a seperate file so i can keep track of which sample goes with which
import os
os.getcwd()
# load in second part as compatible distance matrix 

'/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/CBC_metagenomics/mcav1'

In [29]:
import pandas as pd
import numpy as np

# Read the first 20 lines separately
with open(file_path, 'r') as file:
    first_20_lines = [file.readline().strip() for _ in range(21)]

# Load the distance matrix directly into a NumPy array
# change 'x' value to nan so numpy can read it



def convert_xxxx_to_float(s):
    if s == 'x.xxx':
        return 0.0
    try:
        return float(s)
    except (ValueError, TypeError):
        return 0.0

# Specify the data type and the converter for the columns
dtype = float
converters = {i: convert_xxxx_to_float for i in range(1, 22)}  # Assuming you want to apply the converter to columns 1 to 21

# Skip the first 21 + 1 lines (header lines + "x" line)
skip_rows = 21

# Load the data using numpy
data = np.genfromtxt(file_path, delimiter='\t', skip_header=skip_rows, dtype=dtype, converters=converters)

# Print the modified data array
print(data)
#b_diversity = np.genfromtxt(file_path, skiprows=21, dtype=int,converters = {3: lambda s: float(s or 0)}.)

# Read the remaining lines into a DataFrame
#b_diversity = pd.read_csv(file_path, sep='\t', skiprows=21, header=0, index_col=0)


# Display the distance matrix
#print(b_diversity)


[[   nan  0.     1.     2.     3.     4.     5.     6.     7.     8.
   9.    10.    11.    12.    13.    14.    15.    16.    17.    18.
  19.    20.   ]
 [ 0.     0.     0.453  0.225  0.426  0.18   0.414  0.5    0.466  0.33
   0.471  0.262  0.31   0.163  0.413  0.345  0.39   0.501  0.5    0.243
   0.459  0.251]
 [ 1.     0.     0.     0.437  0.628  0.472  0.669  0.625  0.692  0.394
   0.63   0.523  0.561  0.506  0.688  0.513  0.634  0.73   0.715  0.563
   0.677  0.524]
 [ 2.     0.     0.     0.     0.524  0.249  0.372  0.364  0.407  0.343
   0.369  0.378  0.227  0.231  0.44   0.201  0.384  0.454  0.439  0.35
   0.425  0.291]
 [ 3.     0.     0.     0.     0.     0.439  0.588  0.667  0.636  0.399
   0.588  0.453  0.534  0.428  0.549  0.585  0.489  0.633  0.657  0.373
   0.449  0.48 ]
 [ 4.     0.     0.     0.     0.     0.     0.35   0.451  0.379  0.328
   0.436  0.279  0.233  0.215  0.338  0.273  0.33   0.444  0.435  0.19
   0.415  0.157]
 [ 5.     0.     0.     0.     0.     0.   

In [46]:
#replace null values with 'NaN'
bd_cleaned=b_diversity.replace('x.xxx', 0)
print(bd_cleaned)

        0      1      2      3      4      5      6      7      8      9  ...  \
x                                                                         ...   
0   0.000  0.004  0.109  0.419  0.096  0.151  0.213  0.140  0.104  0.247  ...   
1       0  0.000  0.107  0.422  0.100  0.155  0.211  0.144  0.107  0.244  ...   
2       0      0  0.000  0.505  0.203  0.255  0.107  0.245  0.210  0.142  ...   
3       0      0      0  0.000  0.337  0.289  0.580  0.300  0.330  0.602  ...   
4       0      0      0      0  0.000  0.056  0.302  0.045  0.009  0.334  ...   
5       0      0      0      0      0  0.000  0.350  0.012  0.050  0.382  ...   
6       0      0      0      0      0      0  0.000  0.340  0.309  0.038  ...   
7       0      0      0      0      0      0      0  0.000  0.039  0.372  ...   
8       0      0      0      0      0      0      0      0  0.000  0.341  ...   
9       0      0      0      0      0      0      0      0      0  0.000  ...   
10      0      0      0     

In [19]:
from skbio import DistanceMatrix
from skbio.stats.ordination import pcoa
#b_pc = pcoa(bd_cleaned)
b_diversity

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
x,,,,,,,,,,,,,,,,,,,,,
0,0.000,0.453,0.225,0.426,0.180,0.414,0.500,0.466,0.330,0.471,...,0.310,0.163,0.413,0.345,0.390,0.501,0.500,0.243,0.459,0.251
1,x.xxx,0.000,0.437,0.628,0.472,0.669,0.625,0.692,0.394,0.630,...,0.561,0.506,0.688,0.513,0.634,0.730,0.715,0.563,0.677,0.524
2,x.xxx,x.xxx,0.000,0.524,0.249,0.372,0.364,0.407,0.343,0.369,...,0.227,0.231,0.440,0.201,0.384,0.454,0.439,0.350,0.425,0.291
3,x.xxx,x.xxx,x.xxx,0.000,0.439,0.588,0.667,0.636,0.399,0.588,...,0.534,0.428,0.549,0.585,0.489,0.633,0.657,0.373,0.449,0.480
4,x.xxx,x.xxx,x.xxx,x.xxx,0.000,0.350,0.451,0.379,0.328,0.436,...,0.233,0.215,0.338,0.273,0.330,0.444,0.435,0.190,0.415,0.157
5,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,0.000,0.238,0.126,0.540,0.432,...,0.230,0.376,0.118,0.327,0.363,0.137,0.171,0.381,0.351,0.287
6,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,0.000,0.217,0.549,0.398,...,0.293,0.494,0.312,0.278,0.494,0.265,0.218,0.528,0.427,0.387
7,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,0.000,0.570,0.407,...,0.238,0.433,0.201,0.272,0.423,0.200,0.150,0.450,0.355,0.320
8,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,0.000,0.535,...,0.426,0.338,0.549,0.429,0.414,0.608,0.599,0.394,0.528,0.393


In [17]:
bd2=np.transpose(b_diversity)
bd2

x,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.000,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
1,0.453,0.000,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
2,0.225,0.437,0.000,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
3,0.426,0.628,0.524,0.000,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
4,0.180,0.472,0.249,0.439,0.000,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
5,0.414,0.669,0.372,0.588,0.350,0.000,x.xxx,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
6,0.500,0.625,0.364,0.667,0.451,0.238,0.000,x.xxx,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
7,0.466,0.692,0.407,0.636,0.379,0.126,0.217,0.000,x.xxx,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
8,0.330,0.394,0.343,0.399,0.328,0.540,0.549,0.570,0.000,x.xxx,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
9,0.471,0.630,0.369,0.588,0.436,0.432,0.398,0.407,0.535,0.000,...,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx,x.xxx
